In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Uploading data
url = "https://axe.inline-ltd.ru/data/meatinfo.csv"
data = pd.read_csv(url, sep=';')

# Let's filter out classes with less than 500 examples
product_counts = data['mtype'].value_counts()
valid_products = product_counts[product_counts >= 500].index
data = data[data['mtype'].isin(valid_products)]

# Reduce the size of the data for debugging
sample_data = data.sample(5000, random_state=42)

# Dividing the data into training and test samples
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sample_data['text'], 
    sample_data['mtype'],
    test_size=0.2,
    random_state=42
)

# Encoding of labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)
num_classes = len(label_encoder.classes_)

In [ ]:
# Creating a custom Dataset and DataLoader
class MeatDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initializing the tokenizer and creating a DataLoader
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
max_len = 128

train_dataset = MeatDataset(train_texts.values, train_labels, tokenizer, max_len)
test_dataset = MeatDataset(test_texts.values, test_labels, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Model definition and training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=num_classes)
model = model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Model Training
epochs = 4

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

In [ ]:
# Evaluation of the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))

In [ ]:
# Testing on new examples
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, pred = torch.max(outputs.logits, dim=1)
    
    return label_encoder.inverse_transform(pred.cpu().numpy())[0]

examples = [
    "Говядина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную говядину 2 сорт (80/20) Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.",
    "Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:",
    "Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.",
]

for example in examples:
    print(f'Text: {example}\nPredicted Product: {predict(example)}\n')